In [4]:
import pandas as pd
from datetime import datetime


final_pdf_file = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\final_pdf_file.xlsx')
final_combined_properties_data = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\final_combined_properties_data.xlsx')
final_combined_contacts_data = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\final_combined_contacts_data.xlsx')
owner_properties = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\owner_properties.xlsx')
tenant_properties = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\tenant_properties.xlsx')


property_docs = pd.DataFrame(columns=[
    'id', 'property_id', 'doc_path', 'created_at', 'updated_at', 'contact_id', 'owner_id', 
    'tenant_id', 'name', 'file_size', 'generated', 'supplier_contact_id', 'buyer_contact_id', 
    'seller_contact_id', 'company_id', 'language_code'
])


timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


id_counter = 100


new_rows = []


final_combined_contacts_data['Roles'] = final_combined_contacts_data['Roles'].str.strip().str.lower()


for index, row in final_pdf_file.iterrows():
    property_name = row['Property Name']
    pdf_property_id = row['pdf_property_id']
    
    
    property_data = final_combined_properties_data[final_combined_properties_data['Reference'] == property_name]
    property_id = property_data['properties_row'].values[0] if not property_data.empty else None
    
    
    contact_data = final_combined_contacts_data[final_combined_contacts_data['Inner_Property'] == property_name]
    contact_id = contact_data['Row No'].values[0] if not contact_data.empty else None
    
    
    owner_data = owner_properties[owner_properties['property_id'] == pdf_property_id]
    owner_id = owner_data['id'].values[0] if not owner_data.empty else None
    
    
    tenant_data = tenant_properties[tenant_properties['property_id'] == pdf_property_id]
    tenant_id = tenant_data['id'].values[0] if not tenant_data.empty else None
    
   
    supplier_data = final_combined_contacts_data[
        (final_combined_contacts_data['Inner_Property'] == property_name) & 
        (final_combined_contacts_data['Roles'].isin(['supplier', 'supplier, tenant', 'owner, supplier', 
                                                     'owner, supplier, tenant', 'seller, supplier']))
    ]
    supplier_contact_id = supplier_data['Row No'].values[0] if not supplier_data.empty else None
    
    
    buyer_contact_data = final_combined_contacts_data[final_combined_contacts_data['Inner_Property (Buyer)'] == property_name]
    buyer_contact_id = buyer_contact_data['Row No'].values[0] if not buyer_contact_data.empty else None

    
    seller_data = final_combined_contacts_data[
        (final_combined_contacts_data['Inner_Property'] == property_name) & 
        (final_combined_contacts_data['Roles'].isin(['seller', 'supplier, tenant', 'owner, seller', 'seller, supplier']))
    ]
    seller_contact_id = seller_data['Row No'].values[0] if not seller_data.empty else None
    
    
    new_row = {
        'id': id_counter,
        'property_id': property_id,
        'doc_path': row['modified link'],
        'created_at': timestamp,
        'updated_at': timestamp,
        'contact_id': contact_id,
        'owner_id': owner_id,
        'tenant_id': tenant_id,
        'name': row['pdf name'],
        'file_size': row['file size'],
        'generated': None,
        'supplier_contact_id': supplier_contact_id,
        'buyer_contact_id': buyer_contact_id,
        'seller_contact_id': seller_contact_id,
        'company_id': 1,
        'language_code': None
    }
    
    
    if any(pd.notna(val) for val in new_row.values()):
        new_rows.append(new_row)
    
    
    id_counter += 1


if new_rows:
    new_df = pd.DataFrame(new_rows)
    
    new_df = new_df.dropna(how='all')

    
    if not new_df.empty:
        property_docs = pd.concat([property_docs, new_df], ignore_index=True)


output_path = r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\property_docs.xlsx'
property_docs.to_excel(output_path, index=False)

print(f"property_docs.xlsx file has been created at {output_path}")


property_docs.xlsx file has been created at C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\property_docs.xlsx


C:\Users\MSI\AppData\Local\Temp\ipykernel_34216\3030184286.py:105: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  property_docs = pd.concat([property_docs, new_df], ignore_index=True)
